## 6.5 Pooling层
双重目的:
+ 降低卷积层对位置的敏感性
+ 降低对空间采样表示的敏感性



### 6.5.1 最大汇聚层和平均汇聚层
汇聚层运算符由一个固定形状的窗口组成，该窗口根据其步幅大小在输入的所有区域上滑动，为固定形状窗口（有时称为汇聚窗口）遍历的每个位置计算一个输出。汇聚层不包含参数，**池运算通常是计算窗口中所有元素的最大值或平均值**

![最大汇聚层](https://cdn.staticaly.com/gh/SisyphusTang/Picture-bed@master/20230812/最大汇聚层.jh6frlb6rew.webp)

总结:

对于给定输入元素，最大汇聚层会输出该窗口内的最大值，平均汇聚层会输出该窗口内的平均值。

汇聚层的主要优点之一是减轻卷积层对位置的过度敏感。

我们可以指定汇聚层的填充和步幅。

使用最大汇聚层以及大于1的步幅，可减少空间维度（如高度和宽度）。

汇聚层的输出通道数与输入通道数相同。

汇聚窗口形状为 p × q 的汇聚层称为p × q汇聚层，汇聚操作称为p × q汇聚


In [7]:
import torch
from torch import nn
from d2l import torch as d2l


def pool2d(X, pool_size, mode = 'max'):
    p_h,p_w = pool_size

    Y = torch.zeros((X.shape[0] - p_h + 1), (X.shape[1] - p_w + 1))
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j] = X[i:i+p_h, j:j+p_w].max()
            elif mode == 'avg':
                Y[i,j] = X[i:i+p_h, j:j+p_w].mean()
    
    return Y


In [9]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [8]:
pool2d(X, (2,2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

填充和步幅

池化层也可以改变输出形状，可以通过填充或步幅以获取所需的输出形状

In [20]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [11]:
# 默认步幅和窗口大小相同 这里步幅也和窗口大小相同为3×3
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

In [12]:
# 填充和步幅可以手动设定
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [13]:
# 任意大小的矩形汇聚窗口，并分别设定填充和步幅的高度和宽度
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

多个通道
在处理多通道输入数据时，汇聚层在每个输入通道上单独运算，而不是像卷积层一样在通道上对输入进行汇总。这意味着汇聚层的输出通道数与输入通道数相同。 

下面的代码汇聚后输出通道数量仍然是2


In [22]:
X[0,0,:,:]

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]])

In [23]:
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [24]:
X[0,:,:,:]

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.]],

        [[ 1.,  2.,  3.,  4.],
         [ 5.,  6.,  7.,  8.],
         [ 9., 10., 11., 12.],
         [13., 14., 15., 16.]]])

In [25]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])